In [1]:
import pandas as pd
import os

thedir = os.getcwd()
writedir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/interim'))
extdir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/external'))

In [2]:

maison_blanche_mean = [8.5,10.7,14.3,15.1,17.3,"nan","nan"]
vogelsang_mean = [10.3,12.2,16.1,"nan","nan",14.7,9.4]
boezingenmoos_mean = [10.2,12.1,16.1,16.5,17.9,14.5,9.2]
maison_min = [-1.2,0.6,6.5,6.2,6.2,3.1,'nan']
vogel_min = [0.7,2.5,3.7,8.6,'nan',4.4,0.9]
boz_min = [-1.9,0.6,6.5,6.2,6.2,3.1,-0.9]
maison_max = [18.9,22.3,25.5,24.4,26.4,20.3,18.4]
vogel_max = [21.8,24.3,27.9,27.7,29.6,22.4,20.5]
boz_max = [22.1,25.3,29.6,28.6,31.2,24.1,21.8,]

attributes_maison = {'project':'archive', 'name':"maison_blanche", 'date_range':('1983-04','1983-10'), 'sensor_type':'archive_fixed','city':'Biel', 'coord':("584'300","21'600"), 
                   'geo_ref':'old swiss - unknown', 'elevation':720,'accuracy':'+/- 0.53', 'resolution':0.14}
attributes_log_1 = {'project':'biel-temps', 'name':"maison_blanche", 'date_range':None, 'sensor_type':'lcd','city':'Biel', 'coord':("2'585'009.153, 1'219'997.855"), 
                   'geo_ref':'SwissTopo', 'elevation':432,'accuracy':'+/- 0.53', 'resolution':0.14}
attributes_os = {'project':'open-sense', 'name':"Gumme Brügg", 'date_range':None, 'sensor_type':'SI','city':'Biel', 'coord':(47.126465,7.285249), 
                   'geo_ref':'SwissTopo', 'elevation':469,'accuracy':'+/- 0.53', 'resolution':0.14}

maison=pd.DataFrame([maison_min,maison_max,maison_blanche_mean],columns = ['apr','may','jun','jul','aug','sep','oct'],index = ['min','max','mean'])
vogel=pd.DataFrame([vogel_min,vogel_max,vogelsang_mean],columns = ['apr','may','jun','jul','aug','sep','oct'],index = ['min','max','mean'])
boz=pd.DataFrame([boz_min,boz_max,boezingenmoos_mean],columns = ['apr','may','jun','jul','aug','sep','oct'],index = ['min','max','mean'])

data_lcd = pd.read_csv(F'{writedir}/alldata.csv')
data_ = data_lcd[["Log_201"]].copy()
data_os = pd.read_csv(F'{extdir}/opensense-temp.csv')

In [3]:
class Sensor():
    def __init__(self, data = None,attributes = None):
        self.attributes = attributes
        self.data = data
    def return_att(self):
        return self.attributes
    def return_data(self):
        return self.data

class OpenSensor(Sensor):
    def __init__(self, data = None,attributes = None):
        super().__init__(data,attributes)

class ArchiveSensor(Sensor):
    def __init__(self, data = None,attributes = None):
        super().__init__(data,attributes)

class LowCostSensor(Sensor):
    def __init__(self, data = None,attributes = None):
        super().__init__(data,attributes)

In [4]:
Log_1 = LowCostSensor(data_, attributes_log_1)
os_1 = ArchiveSensor(data_os,attributes_os)
archive_1 = ArchiveSensor(maison,attributes_maison)

In [5]:
Log_1.attributes

{'project': 'biel-temps',
 'name': 'maison_blanche',
 'date_range': None,
 'sensor_type': 'lcd',
 'city': 'Biel',
 'coord': "2'585'009.153, 1'219'997.855",
 'geo_ref': 'SwissTopo',
 'elevation': 432,
 'accuracy': '+/- 0.53',
 'resolution': 0.14}